In [1]:
import os

In [2]:
pwd

'c:\\Prajesh\\personal\\ML_learning\\my_work\\kubeflow_app\\kubeflow_ML_app\\ML_flow_app\\research'

In [3]:
cd ..

c:\Prajesh\personal\ML_learning\my_work\kubeflow_app\kubeflow_ML_app\ML_flow_app


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    target_col: str

In [5]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            target_col = schema.name
        )
        
        return model_trainer_config

In [12]:
import pandas as pd
import os
from mlFlowProject import logger
from sklearn.ensemble import RandomForestRegressor
# import optuna
import joblib
import numpy as np

In [25]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        x_train = np.array(train_data['Open']).reshape(-1, 1)
        x_test = np.array(test_data['Open']).reshape(-1, 1)
        y_train = np.array(train_data[[self.config.target_col]]).reshape(-1)
        y_test = np.array(test_data[[self.config.target_col]]).reshape(-1)
        
        print(x_train.shape, y_train.shape)
        
        rfr = RandomForestRegressor(
            n_estimators= self.config.n_estimators,
            max_depth=self.config.max_depth
        )
        
        rfr.fit(x_train, y_train)
        
        joblib.dump(rfr, os.path.join(self.config.root_dir, self.config.model_name))

In [26]:
try:
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-08-06 12:14:38,275: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-06 12:14:38,277: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-06 12:14:38,279: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-06 12:14:38,280: INFO: common: created directory at: artifacts]
[2023-08-06 12:14:38,280: INFO: common: created directory at: artifacts/model_trainer]
(5205, 1) (5205,)
